# Shearwater Competitor Information Analysis

Analyzing streamer project timelines and quarterly breakdowns.

In [1]:
import pandas as pd
import datetime
from datetime import datetime as dt_parser
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Load streamer project data
raw_data_filepath = "Streamer Projects - SWG - AI.csv"
streamer_df = pd.read_csv(raw_data_filepath, skiprows=0)

# Display initial structure
print(f"Dataset shape: {streamer_df.shape}")
streamer_df.head()

Dataset shape: (44, 22)


,Survey Name,,TGS Revenue,TGS Cost,SWG Win Rate,SWG Cost,TGS Duration,Day Rate,Activity,Company,...,Client,Country,Complete,Mobilisation Location,Mobilisation Start,Deployment Start,Production Start,Production End,Retrieval End,Demobilisation End
0,Bengal National Seismic Program (NSP) 2D,2D,NaN,NaN,NaN,NaN,NaN,#DIV/0!,NaN,Shearwater,...,Oil India Ltd,India,Yes,Kakinada - INDIA,1/22/2025,2/1/2025,2/4/2025,6/13/2025,6/16/2025,6/25/2025
1,Block 15 OBN,OBN,NaN,NaN,NaN,NaN,NaN,NaN,Node Handling,Shearwater,...,Esso Exploration Angola Block 15 Limited (Exxo...,Angola,Yes,Off Luanda - ANGOLA,8/23/2025,8/24/2025,8/25/2025,10/16/2025,10/16/2025,10/23/2025
2,Block 15 OBN,OBN,NaN,NaN,NaN,NaN,NaN,NaN,Source,Shearwater,...,Esso Exploration Angola Block 15 Limited (Exxo...,Angola,Yes,Off Luanda - ANGOLA,8/22/2025,8/24/2025,9/1/2025,10/5/2025,10/5/2025,10/23/2025
3,Block 20 Kaminho OBN,OBN,NaN,NaN,NaN,NaN,NaN,NaN,Node Layout,Shearwater,...,TotalEnergies,Angola,Yes,At Sea - ANGOLA,4/19/2025,4/19/2025,4/21/2025,7/11/2025,7/11/2025,7/21/2025
4,Block 20 Kaminho OBN,OBN,NaN,NaN,NaN,NaN,NaN,NaN,Source,Shearwater,...,TotalEnergies,Angola,Yes,At Sea - ANGOLA,4/18/2025,4/18/2025,4/25/2025,7/9/2025,7/10/2025,7/21/2025


## Phase Duration Calculations

Calculating time spans for each project phase.

In [3]:
# Convert date columns to datetime objects
date_col_names = ["Mobilisation Start", "Deployment Start", "Production Start", 
                  "Production End", "Retrieval End", "Demobilisation End"]

for col_name in date_col_names:
    streamer_df[col_name] = pd.to_datetime(streamer_df[col_name], format="mixed", errors="coerce")

In [4]:
# Calculate phase durations
streamer_df["Mobilization (days)"] = (streamer_df["Deployment Start"] - streamer_df["Mobilisation Start"]).dt.days
streamer_df["Deployment (days)"] = (streamer_df["Production Start"] - streamer_df["Deployment Start"]).dt.days
streamer_df["Production (days)"] = (streamer_df["Production End"] - streamer_df["Production Start"]).dt.days
streamer_df["Recovery (days)"] = (streamer_df["Retrieval End"] - streamer_df["Production End"]).dt.days
streamer_df["Demobilization (days)"] = (streamer_df["Demobilisation End"] - streamer_df["Retrieval End"]).dt.days
streamer_df["Project Duration"] = (streamer_df["Demobilisation End"] - streamer_df["Mobilisation Start"]).dt.days

# Show calculated columns
phase_cols = ["Survey Name", "Mobilization (days)", "Deployment (days)", "Production (days)", 
              "Recovery (days)", "Demobilization (days)", "Project Duration"]
streamer_df[phase_cols].head(10)

,Survey Name,Mobilization (days),Deployment (days),Production (days),Recovery (days),Demobilization (days),Project Duration
0,Bengal National Seismic Program (NSP) 2D,10.0,3.0,129.0,3.0,9.0,154.0
1,Block 15 OBN,1.0,1.0,52.0,0.0,7.0,61.0
2,Block 15 OBN,2.0,8.0,34.0,0.0,18.0,62.0
3,Block 20 Kaminho OBN,0.0,2.0,81.0,0.0,10.0,93.0
4,Block 20 Kaminho OBN,0.0,7.0,75.0,1.0,11.0,94.0
5,Block 32 Louro / Mostarda 4D OBN,16.0,13.0,70.0,0.0,1.0,100.0
6,Block 32 Louro / Mostarda 4D OBN,5.0,2.0,81.0,0.0,1.0,89.0
7,Block 63 / Block 52 Ironman 3D,29.0,5.0,108.0,4.0,9.0,155.0
8,BP-OSHP-2022/1 3D,3.0,7.0,61.0,4.0,4.0,79.0
9,CI-40 Kossipo OBN,0.0,1.0,24.0,0.0,0.0,25.0


## Quarterly Breakdown Calculation

Determining days per quarter for each project.

In [5]:
def extract_quarter_year(date_val):
    """Extract quarter and year from a date value"""
    if pd.isna(date_val):
        return None
    qtr = (date_val.month - 1) // 3 + 1
    return f"Q{qtr}-{date_val.year}"

def compute_days_in_quarter(start_dt, end_dt, target_quarter_str):
    """Calculate days within a specific quarter"""
    if pd.isna(start_dt) or pd.isna(end_dt) or not target_quarter_str:
        return 0
    
    # Parse quarter string
    qtr_num = int(target_quarter_str[1])
    yr_val = int(target_quarter_str.split("-")[1])
    
    # Define quarter boundaries
    qtr_start_month = (qtr_num - 1) * 3 + 1
    qtr_start = pd.Timestamp(year=yr_val, month=qtr_start_month, day=1)
    
    if qtr_num == 4:
        qtr_end = pd.Timestamp(year=yr_val, month=12, day=31)
    else:
        next_qtr_start = pd.Timestamp(year=yr_val, month=qtr_start_month + 3, day=1)
        qtr_end = next_qtr_start - pd.Timedelta(days=1)
    
    # Calculate overlap
    overlap_start = max(start_dt, qtr_start)
    overlap_end = min(end_dt, qtr_end)
    
    if overlap_start > overlap_end:
        return 0
    
    return (overlap_end - overlap_start).days + 1

In [6]:
# Identify all quarters in dataset
all_quarters = set()
for idx, row in streamer_df.iterrows():
    start = row["Mobilisation Start"]
    end = row["Demobilisation End"]
    if pd.notna(start) and pd.notna(end):
        current = start
        while current <= end:
            qtr_label = extract_quarter_year(current)
            if qtr_label:
                all_quarters.add(qtr_label)
            # Move to next quarter
            month_offset = ((current.month - 1) // 3 + 1) * 3 + 1
            if month_offset > 12:
                current = pd.Timestamp(year=current.year + 1, month=1, day=1)
            else:
                current = pd.Timestamp(year=current.year, month=month_offset, day=1)

quarter_list = sorted(list(all_quarters))
print(f"Total quarters identified: {len(quarter_list)}")
print(f"Quarter range: {quarter_list[0]} to {quarter_list[-1]}")

Total quarters identified: 6
Quarter range: Q1-2025 to Q4-2025


In [7]:
# Build quarterly breakdown for each project
quarterly_records = []

for idx, project_row in streamer_df.iterrows():
    project_name = project_row["Survey Name"]
    mobil_start = project_row["Mobilisation Start"]
    demobil_end = project_row["Demobilisation End"]
    revenue_val = project_row.get("TGS Revenue", None)
    
    if pd.isna(mobil_start) or pd.isna(demobil_end):
        continue
    
    for qtr in quarter_list:
        days_in_qtr = compute_days_in_quarter(mobil_start, demobil_end, qtr)
        if days_in_qtr > 0:
            quarterly_records.append({
                "Project": project_name,
                "Quarter": qtr,
                "Days in Quarter": days_in_qtr,
                "Avg. Day Rate": None,
                "Total Cost": None,
                "Total Revenue": revenue_val
            })

quarterly_breakdown_df = pd.DataFrame(quarterly_records)
print(f"Total project-quarter records: {len(quarterly_breakdown_df)}")
quarterly_breakdown_df.head(15)

Total project-quarter records: 62


,Project,Quarter,Days in Quarter,Avg. Day Rate,Total Cost,Total Revenue
0,Bengal National Seismic Program (NSP) 2D,Q1-2025,69,None,None,None
1,Bengal National Seismic Program (NSP) 2D,Q2-2025,86,None,None,None
2,Block 15 OBN,Q3-2025,39,None,None,None
3,Block 15 OBN,Q4-2025,23,None,None,None
4,Block 15 OBN,Q3-2025,40,None,None,None
5,Block 15 OBN,Q4-2025,23,None,None,None
6,Block 20 Kaminho OBN,Q2-2025,73,None,None,None
7,Block 20 Kaminho OBN,Q3-2025,21,None,None,None
8,Block 20 Kaminho OBN,Q2-2025,74,None,None,None
9,Block 20 Kaminho OBN,Q3-2025,21,None,None,None


## Consolidated Quarterly View

Creating swimlane format with quarters as columns.

In [8]:
# Pivot to swimlane format
quarterly_pivot = quarterly_breakdown_df.pivot_table(
    index="Project",
    columns="Quarter",
    values="Days in Quarter",
    aggfunc="sum",
    fill_value=0
)

print(f"Swimlane dimensions: {quarterly_pivot.shape}")
quarterly_pivot.head(10)

Swimlane dimensions: (22, 6)


Quarter,Q1-2025,Q1-2026,Q2-2025,Q3-2025,Q4-2024,Q4-2025
Project,,,,,,
2D,0,0,0,0,0,25
BP-OSHP-2022/1 3D,59,0,21,0,0,0
Bengal National Seismic Program (NSP) 2D,69,0,86,0,0,0
Block 15 OBN,0,0,0,79,0,46
Block 20 Kaminho OBN,0,0,147,42,0,0
Block 32 Louro / Mostarda 4D OBN,154,0,37,0,0,0
Block 63 / Block 52 Ironman 3D,90,0,20,0,46,0
CI-40 Kossipo OBN,11,0,0,0,15,0
EQ25005 Tyrihans 4D,0,0,0,41,0,0


In [9]:
# Create consolidated quarterly summary
quarter_summary = []

for qtr in quarter_list:
    qtr_data = quarterly_breakdown_df[quarterly_breakdown_df["Quarter"] == qtr]
    
    total_days = qtr_data["Days in Quarter"].sum()
    project_count = len(qtr_data)
    
    # Calculate revenue (handling string format with $ and commas)
    revenue_total = 0
    for rev_val in qtr_data["Total Revenue"].dropna():
        if isinstance(rev_val, str):
            clean_val = rev_val.replace("$", "").replace(",", "").replace(" ", "").strip()
            try:
                revenue_total += float(clean_val)
            except:
                pass
    
    quarter_summary.append({
        "Quarter": qtr,
        "Total Days in Quarter": total_days,
        "Active Projects": project_count,
        "Avg. Day Rate": None,
        "Total Cost": None,
        "Total Revenue": f"" if revenue_total > 0 else None
    })

quarter_summary_df = pd.DataFrame(quarter_summary)
quarter_summary_df

,Quarter,Total Days in Quarter,Active Projects,Avg. Day Rate,Total Cost,Total Revenue
0,Q1-2025,705,13,None,None,None
1,Q1-2026,4,2,None,None,None
2,Q2-2025,687,16,None,None,None
3,Q3-2025,547,12,None,None,None
4,Q4-2024,138,5,None,None,None
5,Q4-2025,354,14,None,None,None


## Export Enhanced Dataset

In [10]:
# Save enhanced project data
streamer_df.to_csv("Enhanced_Streamer_Projects.csv", index=False)
quarterly_breakdown_df.to_csv("Quarterly_Breakdown.csv", index=False)
quarter_summary_df.to_csv("Quarterly_Summary.csv", index=False)
print("Data exported successfully!")

Data exported successfully!
